In [2]:
import tensorflow as tf

In [3]:
%config IPCompleter.use_jedi = False

In [4]:
import numpy as np

In [5]:
from sklearn.preprocessing import MinMaxScaler

In [6]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [7]:
y_train  = y_train.astype(int)
y_test  = y_test.astype(int)
batch_size =len(X_train)

print(X_train.shape, y_train.shape,y_test.shape )

(60000, 28, 28) (60000,) (10000,)


In [8]:
scaler = MinMaxScaler()

In [9]:
n_train_samples, n_train_x, n_train_y = X_train.shape
X_train_d2 = X_train.reshape((n_train_samples,n_train_x*n_train_y))


In [23]:
print(X_train_d2)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [10]:
n_test_samples, test_nx, text_ny = X_test.shape
X_test_d2 = X_test.reshape((n_test_samples,test_nx*text_ny))

In [11]:
# Train
X_train_scaled = scaler.fit_transform(X_train_d2.astype(np.float64))

In [12]:
# test
X_test_scaled = scaler.fit_transform(X_test_d2.astype(np.float64))

In [13]:
feature_columns = [tf.feature_column.numeric_column('x', shape=X_train_scaled.shape[1:])]		

In [14]:
estimator = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[300, 100], 
    n_classes=10, 
    model_dir = './models/DNN')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/DNN', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc8c580c150>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
def get_input_fn(data_df, label_df, num_epochs=None, batch_size=128, shuffle=True, steps=1000):
    def input_fn():
        ds = tf.data.Dataset.from_tensor_slices(({"x": data_df}, label_df)) if label_df is not None else tf.data.Dataset.from_tensor_slices(({"x": data_df}))
        if shuffle:
            ds = ds.shuffle(steps)
        ds = ds.batch(batch_size)
        return ds
    return input_fn 

In [16]:
# Train the estimator
train_input = get_input_fn(X_train_scaled, y_train, batch_size=1000, shuffle=False)

In [17]:
estimator.train(input_fn=train_input, steps=1000)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

In [18]:
eval_input = get_input_fn(X_test_scaled, y_test, batch_size=1000, num_epochs=1, shuffle=False)

In [19]:
estimator.evaluate(input_fn=eval_input, steps=None)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-02-01T22:14:30Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/DNN/model.ckpt-2180
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2021-02-01-22:14:32
INFO:tensorflow:Saving dict for global step 2180: accuracy = 0.8842, average_loss = 0.48995796, global_step = 2180, loss = 0.

{'accuracy': 0.8842,
 'average_loss': 0.48995796,
 'loss': 0.489958,
 'global_step': 2180}

In [20]:
estimator_imp=tf.estimator.DNNClassifier(
    hidden_units=[300, 200, 100], feature_columns=feature_columns, model_dir='./models/DNN3',
    n_classes=10, optimizer='Adagrad', dropout=0.3)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/DNN3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc83befa610>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [21]:
estimator_imp.train(input_fn = train_input,steps=1000) 

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/DNN3/model.ckpt-60
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 60 into ./models/DNN3/model.ckpt.
INFO:tensorflow:loss = 2.3012915, step = 60
INFO:tensorflow:Saving checkpoints for 120 into ./models/DNN3/model.ckpt.
INFO:tenso

In [22]:
estimator_imp.evaluate(input_fn=eval_input, steps=None)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-02-01T22:15:57Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/DNN3/model.ckpt-120
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2021-02-01-22:15:58
INFO:tensorflow:Saving dict for global step 120: accuracy = 0.2969, average_loss = 2.198504, global_step = 120, loss = 2.1985

{'accuracy': 0.2969,
 'average_loss': 2.198504,
 'loss': 2.198504,
 'global_step': 120}